In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from pathlib import Path
from fastai.vision.all import*

In [ ]:
train_path=Path("../input/digit-recognizer/")
image_path=Path("../images")

In [ ]:
Path("../input/digit-recognizer")

In [ ]:
image_path.mkdir(exist_ok=True)

In [ ]:
train_data=pd.read_csv(train_path/'train.csv')
test_data=pd.read_csv(train_path/'test.csv')

In [ ]:
train_data.head()

In [ ]:
for index,row in train_data.iterrows():
    label,digit=row[0],row[1:]
    filepath = image_path / "train" / str(label)
    filepath.mkdir(parents=True, exist_ok=True)
    filename = f"{index}.jpg"
    
    digit = digit.values
    digit = digit.reshape(28,28)
    digit = digit.astype(np.uint8)
    
    img = Image.fromarray(digit)
    img.save(filepath/filename)
    

In [ ]:
for index,row in test_data.iterrows():
    
    filepath = image_path / "test" 
    filepath.mkdir(parents=True, exist_ok=True)
    filename = f"{index+1}.jpg"
    
    digit = row.values
    digit = digit.reshape(28,28)
    digit = digit.astype(np.uint8)
    
    img = Image.fromarray(digit)
    img.save(filepath/filename)
    

In [ ]:
data = ImageDataLoaders.from_folder(
    path=image_path / "train",
    valid_pct=0.2,
    batch_tfms=[Normalize.from_stats(0.1,0.7)],
    bs=32,
    size=24
)

data.show_batch(max_n=20)



In [ ]:
learn = vision_learner(data, resnet50, metrics=[accuracy, error_rate])
learn.lr_find()



In [ ]:
learn.fine_tune(20,0.001)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()



In [ ]:
interp.plot_top_losses(10)



In [ ]:
test_files = list((image_path / "test").glob("*.jpg"))



In [ ]:
test_dl = learn.dls.test_dl(test_files)

In [ ]:
predictions,_ = learn.get_preds(dl=test_dl)
predictions = np.argmax(predictions,axis=1)



In [ ]:
image_ids = [int(path.stem) for path in test_files]
submission = pd.DataFrame({"ImageId": image_ids, "Label": predictions})
submission.to_csv("submission.csv",index=False)

In [ ]:
nan